In [22]:
from matplotlib import pyplot as plt
import numpy as np
import os.path
shape_size = 48
freq = np.genfromtxt('data/SP_xaxis.txt')
def spec(batch, ran):
    path = 'data/DATA'+str(batch)+'/DATA'+str(batch)+'_sp'
    miss = []
    for i in ran:
        if not os.path.exists(path+str(i)+'.txt'):
            miss.append(i)
    return miss

import meep as mp
import math
import cmath

shape_size = 48
sx, sy, sz = 1, 1, 4
h = 1.25
dpml = 0.5
b_m, c_m = 1.4, 3.54
res = 17
echo = 1000
cell_size = mp.Vector3(sx,sy,sz)
fcen = 0.5
df = 0.2
theta = math.radians(0)
nfreq = 200

# k with correct length (plane of incidence: XZ) 
k = mp.Vector3(math.sin(theta),0,math.cos(theta)).scale(fcen)
def pw_amp(k, x0):
    def _pw_amp(x):
        return cmath.exp(1j * 2 * math.pi * k.dot(x + x0))
    return _pw_amp

def get_spec(batch, index):
    b = np.genfromtxt('data/DATA'+str(batch) +'_sh.txt')
    start = index * shape_size
    xc = b[start:start + shape_size, 0]
    yc = b[start:start + shape_size, 1]
    vertices = [mp.Vector3(xc[0],yc[0])]

    for j in range(1, len(xc) - 1):
        # eliminate duplicate point
        if xc[j] == xc[j - 1] and yc[j] == yc[j - 1]:
            continue
        vertices.append(mp.Vector3(xc[j], yc[j]))
        #print(xc[i][j], yc[i][j])
    # calculate transmission
    bend = get_bend(vertices)
    Ts = []
    st = np.genfromtxt('data/straight.txt')
    for j in range(nfreq):
        Ts = np.append(Ts, bend[j]/st[j])
    return Ts
#     plt.axis('equal')
#     plt.ylim(-0.5, 0.5)
#     plt.xlim(-0.5, 0.5)
#     plt.plot(x, y)
#     plt.fill(x, y)
#     plt.show()
def get_bend(vertices):
    geometry = [mp.Block(size = cell_size, material=mp.Medium(index=b_m)),
                mp.Prism(vertices, 
                         height=h, 
                         material=mp.Medium(index=c_m),
                         center=mp.Vector3()
                        )]
    pml_layers = [mp.PML(thickness=1, direction = mp.Z, side=mp.High),
                  mp.Absorber(thickness=1,direction = mp.Z, side=mp.Low)]
    src_pos = -(sz/2 - dpml - 0.5)
    src = [mp.Source(src = mp.GaussianSource(fcen, fwidth=df),
                     component = mp.Ey,
                     center = mp.Vector3(0,0,src_pos),
                     size = mp.Vector3(sx,sy,0),
                     amp_func=pw_amp(k,mp.Vector3(0,0,src_pos)))]
    sim = mp.Simulation(resolution=res,
                        cell_size=cell_size,
                        boundary_layers=pml_layers,
                        sources=src,
                        geometry=geometry,
                        k_point=k)
    freg = mp.FluxRegion(center=mp.Vector3(0,0,-src_pos),
                         size = mp.Vector3(sx,sy,0))
    trans = sim.add_flux(fcen, df, nfreq, freg)
    sim.run(until = echo)
    bend = mp.get_fluxes(trans)
    return bend

In [ ]:
batch = 64
ran = [i for i in range(0, 781)]
miss = spec(batch, ran)
for i in miss:
    print('Batch: '+str(batch)+ ' sample: '+str(i))
    Ts = get_spec(batch,i)
    path = 'data/DATA'+str(batch)+'/DATA'+str(batch)+'_sp'
    np.savetxt(path +str(i)+'.txt', Ts)

Batch: 64 sample: 501
-----------
Initializing structure...
Meep: using complex fields.
Meep progress: 11.882352941176471/1000.0 = 1.2% done in 4.0s, 332.9s to go
Meep progress: 24.147058823529413/1000.0 = 2.4% done in 8.0s, 323.8s to go
Meep progress: 36.3235294117647/1000.0 = 3.6% done in 12.0s, 318.7s to go
Meep progress: 49.294117647058826/1000.0 = 4.9% done in 16.0s, 309.0s to go
Meep progress: 62.61764705882353/1000.0 = 6.3% done in 20.0s, 299.8s to go
Meep progress: 75.79411764705883/1000.0 = 7.6% done in 24.0s, 293.0s to go
Meep progress: 89.02941176470588/1000.0 = 8.9% done in 28.0s, 286.8s to go
Meep progress: 102.17647058823529/1000.0 = 10.2% done in 32.0s, 281.5s to go
Meep progress: 115.20588235294117/1000.0 = 11.5% done in 36.0s, 276.8s to go
Meep progress: 128.58823529411765/1000.0 = 12.9% done in 40.0s, 271.3s to go
Meep progress: 141.97058823529412/1000.0 = 14.2% done in 44.0s, 266.2s to go
Meep progress: 154.91176470588235/1000.0 = 15.5% done in 48.0s, 262.1s to go
Me

Meep progress: 327.4705882352941/1000.0 = 32.7% done in 116.2s, 238.6s to go
Meep progress: 337.3235294117647/1000.0 = 33.7% done in 120.2s, 236.1s to go
Meep progress: 348.5/1000.0 = 34.9% done in 124.2s, 232.2s to go
Meep progress: 359.9117647058824/1000.0 = 36.0% done in 128.2s, 228.0s to go
Meep progress: 371.20588235294116/1000.0 = 37.1% done in 132.2s, 223.9s to go
Meep progress: 382.3235294117647/1000.0 = 38.2% done in 136.2s, 220.0s to go
Meep progress: 393.88235294117646/1000.0 = 39.4% done in 140.2s, 215.8s to go
Meep progress: 405.29411764705884/1000.0 = 40.5% done in 144.2s, 211.6s to go
Meep progress: 416.52941176470586/1000.0 = 41.7% done in 148.2s, 207.6s to go
Meep progress: 428.0882352941176/1000.0 = 42.8% done in 152.2s, 203.4s to go
Meep progress: 439.3235294117647/1000.0 = 43.9% done in 156.2s, 199.4s to go
Meep progress: 450.70588235294116/1000.0 = 45.1% done in 160.2s, 195.3s to go
Meep progress: 462.2647058823529/1000.0 = 46.2% done in 164.2s, 191.0s to go
Meep p

Meep progress: 538.0588235294117/1000.0 = 53.8% done in 192.3s, 165.1s to go
Meep progress: 549.3529411764706/1000.0 = 54.9% done in 196.3s, 161.0s to go
Meep progress: 560.7647058823529/1000.0 = 56.1% done in 200.3s, 156.9s to go
Meep progress: 572.0/1000.0 = 57.2% done in 204.3s, 152.9s to go
Meep progress: 583.1764705882352/1000.0 = 58.3% done in 208.3s, 148.9s to go
Meep progress: 594.2058823529412/1000.0 = 59.4% done in 212.3s, 145.0s to go
Meep progress: 605.3823529411765/1000.0 = 60.5% done in 216.3s, 141.0s to go
Meep progress: 616.6470588235294/1000.0 = 61.7% done in 220.3s, 137.0s to go
Meep progress: 628.2058823529412/1000.0 = 62.8% done in 224.3s, 132.8s to go
Meep progress: 639.5882352941177/1000.0 = 64.0% done in 228.3s, 128.7s to go
Meep progress: 651.0588235294117/1000.0 = 65.1% done in 232.3s, 124.5s to go
Meep progress: 662.5294117647059/1000.0 = 66.3% done in 236.3s, 120.4s to go
Meep progress: 673.8823529411765/1000.0 = 67.4% done in 240.3s, 116.3s to go
Meep progre

Meep progress: 771.1176470588235/1000.0 = 77.1% done in 268.3s, 79.6s to go
Meep progress: 782.6764705882352/1000.0 = 78.3% done in 272.3s, 75.6s to go
Meep progress: 794.3529411764706/1000.0 = 79.4% done in 276.3s, 71.5s to go
Meep progress: 805.9117647058823/1000.0 = 80.6% done in 280.3s, 67.5s to go
Meep progress: 817.6764705882352/1000.0 = 81.8% done in 284.3s, 63.4s to go
Meep progress: 828.9411764705882/1000.0 = 82.9% done in 288.4s, 59.5s to go
Meep progress: 840.4705882352941/1000.0 = 84.0% done in 292.4s, 55.5s to go
Meep progress: 851.7058823529412/1000.0 = 85.2% done in 296.4s, 51.6s to go
Meep progress: 863.1764705882352/1000.0 = 86.3% done in 300.4s, 47.6s to go
Meep progress: 875.0294117647059/1000.0 = 87.5% done in 304.4s, 43.5s to go
Meep progress: 886.7647058823529/1000.0 = 88.7% done in 308.4s, 39.4s to go
Meep progress: 898.5882352941177/1000.0 = 89.9% done in 312.4s, 35.3s to go
Meep progress: 910.3235294117646/1000.0 = 91.0% done in 316.4s, 31.2s to go
Meep progres

Meep progress: 946.8235294117646/1000.0 = 94.7% done in 348.5s, 19.6s to go
Meep progress: 958.2941176470588/1000.0 = 95.8% done in 352.5s, 15.3s to go
Meep progress: 969.4705882352941/1000.0 = 96.9% done in 356.5s, 11.2s to go
Meep progress: 980.6470588235294/1000.0 = 98.1% done in 360.5s, 7.1s to go
Meep progress: 991.7647058823529/1000.0 = 99.2% done in 364.5s, 3.0s to go
run 0 finished at t = 1000.0 (34000 timesteps)
Batch: 64 sample: 515
-----------
Initializing structure...
Meep: using complex fields.
Meep progress: 10.470588235294118/1000.0 = 1.0% done in 4.0s, 379.0s to go
Meep progress: 21.294117647058822/1000.0 = 2.1% done in 8.0s, 368.5s to go
Meep progress: 32.3235294117647/1000.0 = 3.2% done in 12.0s, 360.1s to go
Meep progress: 43.205882352941174/1000.0 = 4.3% done in 16.0s, 355.0s to go
Meep progress: 54.76470588235294/1000.0 = 5.5% done in 20.0s, 345.8s to go
Meep progress: 66.23529411764706/1000.0 = 6.6% done in 24.0s, 338.9s to go
Meep progress: 77.79411764705883/1000

Meep progress: 110.05882352941177/1000.0 = 11.0% done in 40.1s, 323.9s to go
Meep progress: 121.32352941176471/1000.0 = 12.1% done in 44.1s, 319.1s to go
Meep progress: 132.2941176470588/1000.0 = 13.2% done in 48.1s, 315.3s to go
Meep progress: 143.38235294117646/1000.0 = 14.3% done in 52.1s, 311.1s to go
Meep progress: 154.64705882352942/1000.0 = 15.5% done in 56.1s, 306.5s to go
Meep progress: 165.8235294117647/1000.0 = 16.6% done in 60.1s, 302.2s to go
Meep progress: 176.94117647058823/1000.0 = 17.7% done in 64.1s, 298.1s to go
Meep progress: 188.11764705882354/1000.0 = 18.8% done in 68.1s, 293.9s to go
Meep progress: 199.3235294117647/1000.0 = 19.9% done in 72.1s, 289.6s to go
Meep progress: 210.5/1000.0 = 21.1% done in 76.1s, 285.5s to go
Meep progress: 221.11764705882354/1000.0 = 22.1% done in 80.1s, 282.2s to go
Meep progress: 232.6764705882353/1000.0 = 23.3% done in 84.1s, 277.4s to go
Meep progress: 244.41176470588235/1000.0 = 24.4% done in 88.1s, 272.5s to go
Meep progress: 2

Meep progress: 355.8235294117647/1000.0 = 35.6% done in 112.1s, 203.0s to go
Meep progress: 368.88235294117646/1000.0 = 36.9% done in 116.1s, 198.7s to go
Meep progress: 382.1470588235294/1000.0 = 38.2% done in 120.1s, 194.2s to go
Meep progress: 395.02941176470586/1000.0 = 39.5% done in 124.1s, 190.1s to go
Meep progress: 408.1470588235294/1000.0 = 40.8% done in 128.1s, 185.8s to go
Meep progress: 421.3235294117647/1000.0 = 42.1% done in 132.2s, 181.5s to go
Meep progress: 434.3529411764706/1000.0 = 43.4% done in 136.2s, 177.3s to go
Meep progress: 446.8235294117647/1000.0 = 44.7% done in 140.2s, 173.5s to go
Meep progress: 459.9117647058823/1000.0 = 46.0% done in 144.2s, 169.3s to go
Meep progress: 473.2352941176471/1000.0 = 47.3% done in 148.2s, 164.9s to go
Meep progress: 486.5/1000.0 = 48.6% done in 152.2s, 160.6s to go
Meep progress: 498.7647058823529/1000.0 = 49.9% done in 156.2s, 156.9s to go
Meep progress: 511.88235294117646/1000.0 = 51.2% done in 160.2s, 152.7s to go
Meep pro

Meep progress: 752.0/1000.0 = 75.2% done in 232.3s, 76.6s to go
Meep progress: 765.0294117647059/1000.0 = 76.5% done in 236.3s, 72.6s to go
Meep progress: 777.8529411764706/1000.0 = 77.8% done in 240.3s, 68.6s to go
Meep progress: 790.7058823529412/1000.0 = 79.1% done in 244.3s, 64.7s to go
Meep progress: 803.9411764705882/1000.0 = 80.4% done in 248.3s, 60.6s to go
Meep progress: 816.9705882352941/1000.0 = 81.7% done in 252.3s, 56.5s to go
Meep progress: 829.8823529411765/1000.0 = 83.0% done in 256.3s, 52.5s to go
Meep progress: 843.0588235294117/1000.0 = 84.3% done in 260.3s, 48.5s to go
Meep progress: 856.2352941176471/1000.0 = 85.6% done in 264.3s, 44.4s to go
Meep progress: 868.7647058823529/1000.0 = 86.9% done in 268.3s, 40.5s to go
Meep progress: 881.5588235294117/1000.0 = 88.2% done in 272.3s, 36.6s to go
Meep progress: 894.4117647058823/1000.0 = 89.4% done in 276.3s, 32.6s to go
Meep progress: 907.1176470588235/1000.0 = 90.7% done in 280.3s, 28.7s to go
Meep progress: 919.52941